Tabular Classification

In [ ]:
!pip install opendatasets --quiet
import opendatasets as od
od.download("https://www.kaggle.com/datasets/mssmartypants/rice-type-classification")

In [ ]:
import os
os.listdir("/content/rice-type-classification")


In [ ]:
import pandas as pd # Data reading and preprocessing


In [ ]:
data_df = pd.read_csv("/content/rice-type-classification/riceClassification.csv")
data_df.head()


In [ ]:
data_df.dropna(inplace=True)
data_df.drop(["id"], axis=1, inplace= True)  # Drop Id column # axis=0 row-wise operation # axis=1 column-wise operation       
data_df.head()